## Imports

In [71]:
import sys,os
import json
import pandas as pd
import numpy as np
import logging
import glob
from google.cloud import storage


In [2]:
os.chdir('../')
CWD = os.getcwd()
PATH_DATA = os.path.abspath(os.path.join(CWD,'data'))
KAGGLE_JSON = os.path.abspath(os.path.join(CWD,'credentials/kaggle.json'))
with open(KAGGLE_JSON) as f:
    kaggle_cred = json.load(f)
print(kaggle_cred)
os.environ['KAGGLE_USERNAME'] = kaggle_cred['username']
os.environ['KAGGLE_KEY'] =  kaggle_cred['key']
from kaggle.api.kaggle_api_extended import KaggleApi


{'username': 'vavasya', 'key': '563387b7543d82ddb41791e484caeef3'}


## Kaggle

In [ ]:
bash_command=f"unzip -o {PATH_DATA}/dutch-energy.zip -d {PATH_DATA}&&rm{PATH_DATA}/dutch-energy.zip "


About this directory
Every file is from a network administrator from a specific year.

The columns in each file are:

- **net_manager**: code of the regional network manager
- **purchase_area**: code of the area where the energy is purchased
- **street**: Name of the street
- **zipcode_from and zipcode_to**: 2 columns for the range of zipcodes covered, 4 numbers and 2 letters
- **city**: Name of the city
- **num_connections**: Number of connections in the range of zipcodes
- **delivery_perc**: percentage of the net consumption of electricity or gas. The lower, the more energy was given back to the grid (for example if you have solar panels)
- **perc_of_active_connections**: Percentage of active connections in the zipcode range
- **type_of_connection**: principal type of connection in the zipcode range. For electricity is # fuses X # ampère. For gas is G4, G6, G10, G16, G25
- **type_conn_perc**: percentage of presence of the principal type of connection in the zipcode range
- **annual_consume**: Annual consume. Kwh for electricity, m3 for gas
- **annual_consume_lowtarif_perc**: Percentage of consume during the low tarif hours. From 10 p.m. to 7 a.m. and during weekends.
- **smartmeter_perc**: percentage of smartmeters in the zipcode ranges

In [95]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1473 entries, 0 to 1472
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   net_manager                 1473 non-null   int64         
 1   purchase_area               1473 non-null   int64         
 2   street                      1473 non-null   object        
 3   zipcode_from                1473 non-null   object        
 4   zipcode_to                  1473 non-null   object        
 5   city                        1473 non-null   category      
 6   num_connections             1472 non-null   float64       
 7   delivery_perc               1473 non-null   float64       
 8   perc_of_active_connections  1473 non-null   float64       
 9   type_conn_perc              1472 non-null   float64       
 10  type_of_connection          1472 non-null   category      
 11  annual_consume              1473 non-null   int64       

In [230]:
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
companies = ['stedin', 'liander','enduris', 'enexis','westland-infra','rendo','coteq'] 
column_types_mapping = {'net_manager':'str', 
                        'purchase_area':'str',
                        'street':'str',
                        'zipcode_from':'str',
                        'zipcode_from':'str',
                        'city':'category',
                        'num_connections':'Int16',
                        'delivery_perc':'float',
                        'perc_of_active_connections':'float',
                        'type_conn_perc':'float',
                        'type_of_connection':'str',
                        'annual_consume_lowtarif_perc':'float',
                         'annual_consume':'float',
                        'smartmeter_perc':'float'
                        }

def assign_types(df:pd.DataFrame ,column_types_mapping:dict):
    """Assigning types to dataframe columns"""
    for k,v in column_types_mapping.items():
        try:
            if k in ['type_conn_perc']:
                df[k] = df[k].str.replace(',', '.')
        except:
            pass
        df = df.rename(columns={"ï»¿NETBEHEERDER": "net_manager"})
        try:
            df[k] = df[k].astype(v)
        except Exception as e:
            df[k] = np.nan
            #print(f'---------{k}-----------')
            print(e)
    return df

assign_types(df,column_types_mapping)

,net_manager,purchase_area,street,zipcode_from,zipcode_to,city,num_connections,delivery_perc,perc_of_active_connections,type_conn_perc,type_of_connection,annual_consume,annual_consume_lowtarif_perc,smartmeter_perc,year
0,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,Dorpsstraat,7468CP,7471AA,ENTER,19,89.0,95.0,89.0,1x35,3810.0,89.0,89.0,2020-01-01
1,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AB,7471AB,GOOR,40,100.0,98.0,78.0,1x35,2637.0,95.0,83.0,2020-01-01
2,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AC,7471AC,GOOR,16,100.0,100.0,100.0,1x35,1366.0,100.0,94.0,2020-01-01
3,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AD,7471AE,GOOR,41,93.0,85.0,54.0,3x25,5101.0,93.0,78.0,2020-01-01
4,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,Kerkstraat,7471AG,7471AG,GOOR,14,86.0,100.0,36.0,1x25,4315.0,100.0,93.0,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,Scherpenzeelseweg,7471GR,7471GR,GOOR,15,33.0,100.0,47.0,1x35,2276.0,100.0,100.0,2020-01-01
96,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,Scherpenzeelseweg,7471GS,7471GS,GOOR,21,38.0,100.0,38.0,1x25,2719.0,100.0,86.0,2020-01-01
97,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,Leeuwerikstraat,7471GT,7471GT,GOOR,13,92.0,100.0,69.0,1x35,2616.0,100.0,92.0,2020-01-01
98,Cogas Infra & Beheer BV,Netbeheerder Centraal Overijssel B.V.,Leeuwerikstraat,7471GV,7471GV,GOOR,11,73.0,100.0,73.0,1x35,5044.0,100.0,91.0,2020-01-01


In [242]:
def transformed_to_parquet(year):
    for company in companies:
        all_files = glob.glob(f"{PATH_DATA}/Electricity/{company}*.csv")
        all_files = list(map(os.path.abspath, all_files))
        #print(all_files)
        for file in all_files:          
            if year in file:
                print('uploaded company: ',company, file)
                #print(f"adding column year {year} to {file}")       
                df = pd.read_csv(file, index_col=None, header=0)
                df['year'] = pd.to_datetime(year)
                df = assign_types(df,column_types_mapping)
                #print()
                df.to_parquet(file.replace('.csv','.parquet'))

    

In [245]:
for year in ['2009']:
    transformed_to_parquet(year)
    upload_to_bucket(year)

uploaded company:  stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2009.csv
uploaded company:  liander /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/liander_electricity_01012009.csv
2009

liander_electricity_01012009.parquet
stedin_electricity_2009.parquet



In [231]:
#path = r'/data/Electricity/' 

for company in companies:
    all_files = glob.glob(f"{PATH_DATA}/Electricity/{company}*.csv")
    all_files = list(map(os.path.abspath, all_files))
    #print(all_files)
    for file in all_files:
        print(company, file)
        for year in years:
            if year in file:
                #print(f"adding column year {year} to {file}")       
                df = pd.read_csv(file, index_col=None, header=0)
                df['year'] = pd.to_datetime(year)
                df = assign_types(df,column_types_mapping)
                #print()
                df.to_parquet(file.replace('.csv','.parquet'))

stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2014.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2019.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2009.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2017.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2020.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2013.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2018.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2012.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2010.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/data/Electricity/stedin_electricity_2011.csv
stedin /opt/courses/dezoomcamp/de_zoocamp/Project/

In [224]:
 
df1 = pd.read_csv('data/Electricity/coteq_electricity_2016.csv')
df1

,net_manager,purchase_area,street,zipcode_from,zipcode_to,city,num_connections,delivery_perc,perc_of_active_connections,type_conn_perc,type_of_connection,annual_consume,annual_consume_lowtarif_perc,smartmeter_perc
0,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Dorpsstraat,7468CP,7471AA,ENTER,19,89.47,94.74,89,1x35,4122,89.47,0.0
1,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AB,7471AB,GOOR,37,100.00,100.00,86,1x35,1800,94.59,0.0
2,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AC,7471AC,GOOR,16,100.00,100.00,100,1x35,1315,100.00,0.0
3,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AD,7471AE,GOOR,25,92.00,84.00,44,1x35,6379,92.00,0.0
4,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Kerkstraat,7471AG,7471AG,GOOR,14,85.71,100.00,36,1x35,4404,92.86,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Sterrebos,7609ZP,7609ZP,ALMELO,27,14.81,100.00,96,1x35,2619,100.00,0.0
2499,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Loofgang,7609ZR,7609ZR,ALMELO,28,71.43,100.00,89,1x35,3846,100.00,0.0
2500,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Vista,7609ZS,7609ZS,ALMELO,25,60.00,100.00,76,1x35,5312,96.00,0.0
2501,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Tuinvaas,7609ZT,7609ZT,ALMELO,18,77.78,100.00,94,1x35,4111,100.00,0.0


In [226]:
df1.columns

Index(['net_manager', 'purchase_area', 'street', 'zipcode_from', 'zipcode_to',
       'city', 'num_connections', 'delivery_perc',
       'perc_of_active_connections', 'type_conn_perc', 'type_of_connection',
       'annual_consume', 'annual_consume_lowtarif_perc', 'smartmeter_perc'],
      dtype='object')

## S3

In [13]:
os.environ["GCP_PROJECT_ID"] = "netherlands-energy"
os.environ["GCP_GCS_BUCKET"] = "data_lake_netherlands-energy"
PROJECT_ID = os.environ.get("GCP_PROJECT_ID")
BUCKET = os.environ.get("GCP_GCS_BUCKET")

In [246]:
client = storage.Client()
def upload_to_gcs(bucket, object_name, local_file):
    """
    :param bucket: GCS bucket name
    :param object_name: target path & file-name
    :param local_file: source path & file-name
    :return:
    """
    # WORKAROUND to prevent timeout for files > 6 MB on 800 kbps upload speed.
    # (Ref: https://github.com/googleapis/python-storage/issues/74)
    storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB
    storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
    # End of Workaround 
    bucket = client.bucket(bucket)
    blob = bucket.blob(object_name)
    blob.upload_from_filename(local_file)
    
    
def clean_bucket(bucket):
    bucket = client.bucket(bucket)
    blobs = bucket.list_blobs()
    for blob in blobs:
        blob.delete()

In [244]:
def upload_to_bucket(year):
    clean_bucket(BUCKET)
    print(year)
    print()
    _path = f"{PATH_DATA}/Electricity/*{year}*.parquet"
    files = glob.glob(_path)
    for path_file in files:
        file = os.path.basename(path_file)
        print(file)
        object_name = f"{year}/{file}"
        upload_to_gcs(BUCKET, object_name,path_file)
    print()
            
        
    

In [247]:
clean_bucket(BUCKET)

In [232]:

for year in years:
    print(year)
    print()
    _path = f"{PATH_DATA}/Electricity/*{year}*.parquet"
    files = glob.glob(_path)
    #ll_files = list(map(os.path.abspath, all_files))
    for path_file in files:
        file = os.path.basename(path_file)
        print(file)
        object_name = f"{year}/{file}"
        upload_to_gcs(BUCKET, object_name,path_file)
    print()
            
        
        
        
        
        

2009

liander_electricity_01012009.parquet
stedin_electricity_2009.parquet

2010

stedin_electricity_2010.parquet
enexis_electricity_01012010.parquet
liander_electricity_01012010.parquet

2011

stedin_electricity_2011.parquet
westland-infra_electricity_2011.parquet
enexis_electricity_01012011.parquet
rendo_electricity_2011.parquet
liander_electricity_01012011.parquet

2012

westland-infra_electricity_2012.parquet
stedin_electricity_2012.parquet
rendo_electricity_2012.parquet
liander_electricity_01012012.parquet
enexis_electricity_01012012.parquet

2013

enexis_electricity_01012013.parquet
liander_electricity_01012013.parquet
coteq_electricity_2013.parquet
stedin_electricity_2013.parquet
rendo_electricity_2013.parquet
westland-infra_electricity_2013.parquet
enduriselectricity_01012013.parquet

2014

coteq_electricity_2014.parquet
westland-infra_electricity_2014.parquet
liander_electricity_01012014.parquet
rendo_electricity_2014.parquet
stedin_electricity_2014.parquet
enexis_electricity_

## Bigquery

In [ ]:
 f"gs://{BUCKET}/city_bikes_trips/2018/*.{INPUT_FILETYPE}",
                    f"gs://{BUCKET}/city_bikes_trips/2019/*.{INPUT_FILETYPE}",
                    f"gs://{BUCKET}/city_bikes_trips/2020/*.{INPUT_FILETYPE}",

In [23]:
folders  = []
[folders.append(f"gs://{BUCKET}/{year}/*.parquet")for year in years]
folders

['gs://data_lake_netherlands-energy/2009/*.parquet',
 'gs://data_lake_netherlands-energy/2010/*.parquet',
 'gs://data_lake_netherlands-energy/2011/*.parquet',
 'gs://data_lake_netherlands-energy/2012/*.parquet',
 'gs://data_lake_netherlands-energy/2013/*.parquet',
 'gs://data_lake_netherlands-energy/2014/*.parquet',
 'gs://data_lake_netherlands-energy/2015/*.parquet',
 'gs://data_lake_netherlands-energy/2016/*.parquet',
 'gs://data_lake_netherlands-energy/2017/*.parquet',
 'gs://data_lake_netherlands-energy/2018/*.parquet',
 'gs://data_lake_netherlands-energy/2019/*.parquet',
 'gs://data_lake_netherlands-energy/2020/*.parquet']

In [38]:
# ROJECT_ID = os.environ.get("GCP_PROJECT_ID")
# BUCKET = os.environ.get("GCP_GCS_BUCKET")

path_to_local_home = os.environ.get("AIRFLOW_HOME", "/opt/airflow/")
BIGQUERY_DATASET = os.environ.get("BIGQUERY_DATASET", 'energy_data')

DATASET = "bq_energy_dataset"
PARTITION_COL = 'year'
CLUSTER_COL = 'city'
INPUT_PART = "raw"
INPUT_FILETYPE = "parquet"

In [39]:
f"CREATE OR REPLACE TABLE {BIGQUERY_DATASET}.{DATASET} \
        PARTITION BY DATE({PARTITION_COL}) \
        CLUSTER BY {CLUSTER_COL} AS \
        SELECT * FROM {BIGQUERY_DATASET}.{DATASET}_external_table;"

'CREATE OR REPLACE TABLE energy_data.bq_energy_dataset         PARTITION BY DATE(year)         CLUSTER BY city AS         SELECT * FROM energy_data.bq_energy_dataset_external_table;'

In [ ]:

 
  
bucket_name = ‘BUCKET_NAME’ 
create_bucket(bucket_name)

In [ ]:
          "bucket": BUCKET,
            "object_name": f"city_bikes_trips/{{{{ execution_date.strftime(\'%Y\') }}}}/{parquet_file}",
            "local_file": f"{path_to_local_home}/{parquet_file}",
        },

In [ ]:
upload_to_gcs(bucket, object_name, local_file)

In [ ]:
import os
from datetime import datetime

# from airflow import DAG
# from airflow.operators.bash import BashOperator
# from airflow.operators.python import PythonOperator

#from utils import format_to_parquet, upload_to_gcs




In [ ]:
from datetime import datetime
datetime.now().strftime('%Y%m')

In [ ]:

path_to_local_home = os.environ.get('AIRFLOW_HOME', path_data)

In [87]:
df1.columns

Index(['net_manager', 'purchase_area', 'street', 'zipcode_from', 'zipcode_to',
       'city', 'num_connections', 'delivery_perc',
       'perc_of_active_connections', 'type_conn_perc', 'type_of_connection',
       'annual_consume', 'year'],
      dtype='object')

In [45]:
df.head()

,net_manager,purchase_area,street,zipcode_from,zipcode_to,city,num_connections,delivery_perc,perc_of_active_connections,type_conn_perc,type_of_connection,annual_consume,annual_consume_lowtarif_perc,smartmeter_perc,year
0,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Dorpsstraat,7468CP,7471AA,ENTER,19,89.47,94.74,89,1x35,4122,89.47,0.0,2013-01-01
1,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AB,7471AB,GOOR,37,100.00,100.00,86,1x35,1800,94.59,0.0,2013-01-01
2,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AC,7471AC,GOOR,16,100.00,100.00,100,1x35,1315,100.00,0.0,2013-01-01
3,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,De Stoevelaar,7471AD,7471AE,GOOR,25,92.00,84.00,44,1x35,6379,92.00,0.0,2013-01-01
4,Coteq Netbeheer BV,Netbeheerder Centraal Overijssel B.V.,Kerkstraat,7471AG,7471AG,GOOR,14,85.71,100.00,36,1x35,4404,92.86,0.0,2013-01-01


In [58]:
df['type_of_connection'].unique()

array(['1x35', '3x25', '1x25', '3x63', '3x50', '3x35', '3x80'],
      dtype=object)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2503 entries, 0 to 2502
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   net_manager                   2503 non-null   object        
 1   purchase_area                 2503 non-null   object        
 2   street                        2503 non-null   object        
 3   zipcode_from                  2503 non-null   object        
 4   zipcode_to                    2503 non-null   object        
 5   city                          2503 non-null   object        
 6   num_connections               2503 non-null   int64         
 7   delivery_perc                 2503 non-null   float64       
 8   perc_of_active_connections    2503 non-null   float64       
 9   type_conn_perc                2503 non-null   int64         
 10  type_of_connection            2503 non-null   object        
 11  annual_consume                

In [43]:
df.columns

Index(['net_manager', 'purchase_area', 'street', 'zipcode_from', 'zipcode_to',
       'city', 'num_connections', 'delivery_perc',
       'perc_of_active_connections', 'type_conn_perc', 'type_of_connection',
       'annual_consume', 'annual_consume_lowtarif_perc', 'smartmeter_perc',
       'year'],
      dtype='object')

ValueError: invalid literal for int() with base 10: 'Netbeheerder Centraal Overijssel B.V.'

In [ ]:
pd.read_parquet('../data/Electricity/rendo_electricity_2011.parquet').columns

In [ ]:
all_files = glob.glob(os.path.join('../data/Electricity', "*.parquet"))

df = pd.concat((pd.read_parquet(f) for f in all_files), ignore_index=True)

In [ ]:
df.describe().transpose()

In [ ]:

# iterate over files in
# that directory
for root, dirs, files in os.walk(path_to_local_home):
    for filename in files:       
        #file_relative_path = os.path.join(root.replace(path_data, ''), filename)[1:]
        file_relative_path = os.path.join(root, filename)
        format_to_parquet(file_relative_path)


In [ ]:

PROJECT_ID = os.environ.get("GCP_PROJECT_ID")
BUCKET = os.environ.get("GCP_GCS_BUCKET")

csv_file = '{{ execution_date.strftime(\'%Y%m\') }}-citibike-tripdata.csv'
zip_file = f'{csv_file}.zip'
dataset_url = f'https://s3.amazonaws.com/tripdata/{zip_file}'
path_to_local_home = os.environ.get('AIRFLOW_HOME', '/opt/airflow/')
parquet_file = csv_file.replace('.csv', '.parquet')


default_args = {
    "owner": "airflow",
    "start_date": datetime(2018, 1, 1),
    "end_date": datetime(2020, 12, 31),
    "depends_on_past": False,
    "retries": 1,
}


In [ ]:
# NOTE: DAG declaration - using a Context Manager (an implicit way)
with DAG(
    dag_id="data_ingestion_to_gcs_dag",
    schedule_interval="@monthly",
    default_args=default_args,
 #   depends_on_past=True,
    catchup=True,
    max_active_runs=3,
    tags=['city-bikes'],
) as dag:

    download_zip_file_task = BashOperator(
        task_id="download_zip_file_task",
        bash_command=f"curl -sSLf {dataset_url} > {path_to_local_home}/{zip_file}"
    )

    unzip_dataset_task = BashOperator(
        task_id="unzip_dataset_task",
        bash_command=f"unzip -o {path_to_local_home}/{zip_file} -d {path_to_local_home}"
    )

    format_to_parquet_task = PythonOperator(
        task_id="format_to_parquet_task",
        python_callable=format_to_parquet,
        op_kwargs={
            "src_file": f"{path_to_local_home}/{csv_file}",
        },
    )

    local_to_gcs_task = PythonOperator(
        task_id="local_to_gcs_task",
        python_callable=upload_to_gcs,
        op_kwargs={
            "bucket": BUCKET,
            "object_name": f"city_bikes_trips/{{{{ execution_date.strftime(\'%Y\') }}}}/{parquet_file}",
            "local_file": f"{path_to_local_home}/{parquet_file}",
        },
    )

    clean_up_data_task = BashOperator(
        task_id="clean_up_data_task",
        bash_command=f"rm {path_to_local_home}/{zip_file} {path_to_local_home}/{csv_file}"
                     f" {path_to_local_home}/{parquet_file}"
    )

    download_zip_file_task >> unzip_dataset_task >> format_to_parquet_task >> local_to_gcs_task >> clean_up_data_task
